In [1]:
import os
import pickle
import time


In [2]:
import numpy as np
import pandas as pd

In [3]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut

In [4]:
feature_path = os.path.abspath('../../features/vgg19')

In [5]:
model_save_path = os.path.abspath('../../models/vgg19_svm')

In [6]:
leave_one_out_validation = False

In [7]:
#features are loaded here: 

In [8]:
non_umpire = np.load(f'{feature_path}/model1_vgg19_fc1_features_non_umpire.npy')
umpire = np.load(f'{feature_path}/model1_vgg19_fc1_features_umpire.npy')

In [9]:
X_data = np.append(non_umpire, umpire, axis=0)
labels = X_data[:, (X_data.shape[1]-1)]
train_data = X_data[:, 0:(X_data.shape[1]-1)]

In [10]:
#Train Test Split 80-20

In [19]:
x_train,x_test,y_train,y_test = train_test_split(train_data, labels, test_size=0.2, random_state=157)

In [20]:
#Classifier SVM Linear Kernel 

In [21]:
clf = LinearSVC(C=10)
start_time = time.time()
clf = clf.fit(x_train, y_train)
predictions_tr = (clf.predict(x_test))

In [22]:
#10-Fold Cross-validation Accuracy

In [23]:
scores = cross_val_score(clf, x_train, y_train, cv=10)
print('Training Accuracy: %0.4f (+/- %0.2f)' % (scores.mean(), scores.std() * 2))
print('--- %s seconds ---' % (time.time() - start_time))

Training Accuracy: 0.9623 (+/- 0.10)
--- 3.073284864425659 seconds ---


In [24]:
#jack knife validation

In [25]:
if leave_one_out_validation:
   loo_train_acc = []
   loo = LeaveOneOut()
   for train_index, test_index in loo.split(x_tr):
      X_train, X_test = x_tr[train_index], x_tr[test_index]
      y_train, y_test = y_tr[train_index], y_tr[test_index]
      clf = clf.fit(X_train,y_train)
      predictions = (clf.predict(X_test))
      loo_train_acc.append(accuracy_score(y_test, predictions))

   loo_train_accuracy = np.asarray(loo_train_acc)
   print('LOO Accuracy: %0.4f' % loo_train_accuracy.mean())

In [27]:
test_acc = accuracy_score(y_test, predictions_tr)
print('Test Accuracy: %0.4f' % test_acc)

Test Accuracy: 0.9333


In [28]:
pickle.dump(clf, open(f'{model_save_path}/model1_vgg19_svm.sav', 'wb'))